In [1]:
from IPython.core.display import HTML
# https://stackoverflow.com/questions/32156248/how-do-i-set-custom-css-for-my-ipython-ihaskell-jupyter-notebook
styles = open('./custom_style.css', "r").read()
s = '<style>%s</style>' % styles     
HTML(s)

# CONTENTS

⓪ [ARCHITECTURE](#📐-ARCHITECTURE)

① [PREPROCESSING](#🏭-PREPROCESSING)

② [VISUALIZATIONS](#🖼️-VISUALIZATIONS)

③ [HILLSHADING EXPERIMENTS](#⛰-🗺%EF%B8%8F-HILLSHADING-EXPERIMENTS)

# 📐 ARCHITECTURE

🅐 Example of a pipeline **from an internet**

<table>
  <tr>
    <td><img src="https://ars.els-cdn.com/content/image/1-s2.0-S1574954122002862-gr1.jpg"></td>
  </tr>
  <tr>
    <td><a href="https://www.sciencedirect.com/science/article/pii/S1574954122002862">[Link]</a></td>
  </tr>
</table>

🅑 Pipeline scheme of **the current project**

<table>
    <tr>
      <td>
      <img src="../figures/external/pipeline_flow.png?raw=true" align="center" alt="pipeline flow (draft)" width="75%">
      </td>
    </tr>
</table>

[To table of contents](#CONTENTS)

# 🏭 PREPROCESSING
_From LiDAR Point-Cloud to Mesh Raster_

🅒 Preprocessing scheme of the current project
<table>
  <tr>
    <td><img src="../figures/external/preprocessing_logic.png?raw=true" align="right" alt="preprocessing_logic" width="80%"></td>
  </tr>
</table>

In [ ]:
import sys; sys.path.insert(0, '..')

import src.get_data as get_data
import src.custom_visualizations as custom_visualizations

import importlib; importlib.reload(get_data); importlib.reload(custom_visualizations);

In [ ]:
print('Create folders')
_create_folders_structure()

for key in DATA_LINKS.keys():
    print(f'CITY : {key}')
    
    print('Download')
    get_data.download_lidar_files(links=DATA_LINKS[key], run=False)
    gc.collect()
    
    print('Decompress LAZ')
    if key != 'MIAMI FL USA':
        get_data.decompress_laz_files(input_folder='/kaggle/working/laz_files/', run=False)
        gc.collect()
        
    print('Rasterize (& filter outliers in point clouds)')
    get_data.rasterize_lidar_files(input_folder='/kaggle/working/las_files/', filter_outliers=True, run=False)
    gc.collect()
    
    print('Mosaic')
    mosaic_backend = 'whitebox'
    output_mosaic_path = f'/kaggle/working/{key}_mosaic_{mosaic_backend}_bilinear.tif'
    get_data.mosaic_rastersized_lidar_files(input_folder='/kaggle/working/tif_files/', output_mosaic_path=output_mosaic_path, mosaic_backend=mosaic_backend, run=False)
    gc.collect()
    
    print('Filter raster')
    output_filtered_mosaic_path = f'/kaggle/working/{key}_{mosaic_backend}_bilinear_filtered.tif'
    get_data.filter_mosaiced_raster_file(input_file=output_mosaic_path, output_mosaic_path=output_filtered_mosaic_path, method='median', run=False)
    gc.collect()
    
    break

Check whether unneeded files deleted

In [ ]:
# glob.glob('/kaggle/working/laz_files/*.laz')
# !rm /kaggle/working/laz_files/*.laz

[To table of contents](#CONTENTS)

# 🖼️ VISUALIZATIONS
_From Mesh Raster to 3D Rendered Plots_

* ① **Isometric and orthographic** views (static angles)
* ② **Orbiting** view (fly around)

<table>
  <tr>
    <td><b>Multi-layered render</b><br>[example from VisIt - VTK backed]</td>
    <td><b>Orbiting around Miami, FL</b><br>[internal, PyVista - VTK backed]</td>
    <td><b>Orthographic Miami, FL</b><br>[internal, PyVista - VTK backed]</td>
  </tr>
  <tr>
    <td valign="top"><img src="https://visit-dav.github.io/visit-website/images/gallery-15.jpg" width='500', height="500"></td>
    <td valign="top"><img src="../figures/internal/orbiting/MIAMI FL USA_orbiting_2K_36points_Shades.gif" width='500', height="500">
    <td valign="top"><img src="../figures/internal/low_resolution/MIAMI FL USA_tst1_biliniar_ortho_1600x1600.png" width='500', height="500"></td>
  </tr>
</table>

In [ ]:
%%time
fig, axes = plt.subplots(1, 2, figsize=(8, 10))

titles = ['+ Outliers filter', '+ Median filter']
for indx, file in enumerate(glob.glob('/kaggle/input/miami-beach-lidar-raster/*.tif')):
    axes[indx].set_title(titles[indx])
    custom_visualizations.plot_raster_rasterio(raster_path=file, ax=axes[indx])

fig.tight_layout()
gc.collect()

In [ ]:
%%time
experiment_number = 1
for indx, file in enumerate(glob.glob('/kaggle/input/miami-beach-lidar-raster/*.tif')):
    print(f"{datetime.datetime.now()} Reading for Visualisation. Processing {file}.")
    mesh = get_data._read_processed_rasterized_file(file_path=file, 
                                      CUSTOM_READ=True)
    display(mesh)
    topo_clipped = mesh.clip_scalar(scalars="data", value=0.1, invert=False, progress_bar=True)
    display(topo_clipped)
    if 'mosaic' in file:
        topo_clipped.rotate_z(90) # if Miami
        key = 'MIAMI FL USA'
        wrap_factor = 0.02
    else:
        topo_clipped.rotate_z(-90) # if Dallas
        key = 'DALLAS TX USA'
        wrap_factor = 0.5
    gc.collect()
    print(f'\t{key}')
    
    #if add_shades: Shades_
    # window_size = [1000, 1000]
    print(f"\t{datetime.datetime.now()} Visualisation. Orbiting.")
    plot_filename = f"{key}_tst{experiment_number}_biliniar_orbiting_1k_shades.gif" # png
    custom_visualizations.plot_3D(mesh=topo_clipped.warp_by_scalar(factor=wrap_factor), plotter_params=None, plot_isometric=True, 
            save_plot_mode='save render orbiting', plot_filename=plot_filename)
    gc.collect()
    print(f"\t{datetime.datetime.now()} Visualisation. Isometric.")
    plot_filename = f"{key}_tst{experiment_number}_biliniar_isometric_1k_shades.png"
    custom_visualizations.plot_3D(mesh=topo_clipped.warp_by_scalar(factor=wrap_factor), plotter_params=None, plot_isometric=True, 
            save_plot_mode='save render', plot_filename=plot_filename)
    gc.collect()
    print(f"\t{datetime.datetime.now()} Visualisation. Orthographic.")
    plot_filename = f"{key}_tst{experiment_number}_biliniar_ortho_1k_shades.png"
    custom_visualizations.plot_3D(mesh=topo_clipped.warp_by_scalar(factor=wrap_factor), plotter_params=None, plot_isometric=False, 
            save_plot_mode='save render', plot_filename=plot_filename)
    gc.collect()

[To table of contents](#CONTENTS)

# ⛰ 🗺️ HILLSHADING EXPERIMENTS
_From Mesh Raster to Shaded Elevation Plots_

① There are a few out-of-a-box ways to plot terrain sheded maps in a fast & good-to-go manner:
1. **`matplotlib.colors.LightSource`**
    * "raw" `matplotlib`
    * terrain shader built into `matplotlib`
    * see [[documentation]](https://matplotlib.org/stable/api/_as_gen/matplotlib.colors.LightSource.html) | [[example]](https://matplotlib.org/stable/gallery/specialty_plots/topographic_hillshading.html)
    
    
2. **`rioxarray.plot.imshow` / `rioxarray.plot.imshow.contour`** (uses **`rasterio`** backend)
    * `matplotlib`-based 
    * raster processing & plotting interfaces of `rioxarray`
    * see [[documentation]](https://corteva.github.io/rioxarray/stable/examples/reproject.html) | [[example]](https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/fundamentals-raster-data/open-lidar-raster-python-xarray/)
    
3. **`earthpy`** (`earthpy.spatial`, `earthpy.plot`)
    * `matplotlib`-based 
    * raster processing & plotting interfaces built into `earthpy`
    * see [[documentation]](https://earthpy.readthedocs.io/en/latest/_modules/earthpy/plot.html?highlight=_plot_image) | [[example]](https://earthpy.readthedocs.io/en/latest/gallery_vignettes/plot_dem_hillshade.html)
    
② Auxilary notes:
1. Check whether you work with **`numpy.ndarray`** or **`xarray.core.dataarray.DataArray`** or some other structure
1. Check whether your raster stores **`CRS`** and **`extents`** information
1. To handle polygons (shapes), check f.e. **`gpd`**, **`shapely`**, **`osmnx`** etc
1. To handle some custom water detection, check flood alghoritms **`skimage.segmentation.flood`** and **`OpenCV's floodFill`**

<table>
    
  <tr>
    <td width='25%'>Hillshade with overlayed polygon(s)<br>[raw `matplotlib` & `rasterio` (np.array]</td>
    <td width='25%'>Hillshade with overlayed polygon(s)<br>[`earthpy` & `rioxarray` (xarray.DataArray)]</td>
  </tr>
  <tr>
    <td valign="top"><img src="../figures/internal/hillshading/matplotlib_ rasterio.png" width="350" height="350"></td>
    <td valign="top"><img src="../figures/internal/hillshading/earthpy_rioxarray.png" width=350 height="350"></td>
  </tr>
    
  <tr align="center">
    <td align="center" colspan=3>`Raw` (EPSG:2236) -> `Reprojected Raw` (EPSG:4326) -> `Elevation Countours`<br>[all - `rioxarray.plot`]</td>
  </tr>
  <tr>
    <td valign="top" colspan=3><img src="../figures/internal/hillshading/raw_projected_countours.png" width="900" height="600"></td>
  </tr>
    
</table>

[To table of contents](#CONTENTS)